<a href="https://colab.research.google.com/github/rubanzasilva/RossmanSales/blob/master/rossman_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ROSSMANN SALES SOLUTION

Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

You are provided with historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.

IMPORTS

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
from torch import nn, optim
import seaborn as sns
from pathlib import Path
import PIL
import json
from fastai import *
from fastai.tabular import *
from fastai.vision import *
from fastai.metrics import error_rate


**LOAD KAGGLE DATASET**

In [0]:
! git clone https://github.com/rubanzasilva/external.git

In [6]:
!ls external

README.md  rossmann.zip


In [0]:
!ls

In [7]:
PATH = Path('content/rossmann/')
PATH


PosixPath('content/rossmann')

In [0]:
!ls external

In [0]:
!unzip external/rossmann.zip

**LOAD INTO DATAFRAME**

In [8]:
PATH=Path('rossmann/')
table_names = ['train', 'store', 'store_states', 'state_names', 'googletrend', 'weather', 'test']
tables = [pd.read_csv(PATH/f'{fname}.csv', low_memory=False) for fname in table_names]
train, store, store_states, state_names, googletrend, weather, test = tables
len(train),len(test)

(1017209, 41088)

In [9]:
for t in tables: display(t.head())

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


,Store,State
0,1,HE
1,2,TH
2,3,NW
3,4,BE
4,5,SN


,StateName,State
0,BadenWuerttemberg,BW
1,Bayern,BY
2,Berlin,BE
3,Brandenburg,BB
4,Bremen,HB


,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67


,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,11.0,5.0,2.0,23,14,NaN,0.25,7.0,Rain,263
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,10.0,6.0,3.0,16,10,NaN,0.00,7.0,Rain,268


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


**DATA CLEAN**

In [0]:
#turn state holidays to boolean
train.StateHoliday = train.StateHoliday!= '0'
test.StateHoliday = test.StateHoliday!= '0'

join tables

In [0]:
def join_df(left , right , left_on , right_on=None , suffix='_y'):
  if right_on is None: right_on = left_on
  return left.merge(right , how='left' , left_on = left_on , right_on = right_on , suffixes=("", suffix))

In [0]:
#join weather and state names
weather = join_df(weather, state_names , "file" , "StateName")

In [0]:
#extract dates and states names and add their columsn to googletrends
googletrend['Date'] =  googletrend.week.str.split('-' , expand = True)[0]
googletrend['State'] = googletrend.file.str.split('_' , expand = True)[2]
googletrend.loc[googletrend.State == 'NI' , "State" ] = ' HB ,NI '

Extract Date Fields

In [0]:
def add_datepart(df , fieldName , drop= True , time= False ):
  #helper function to add columns relevant to date
  field = df[fieldName]
  field_dtype = field.dtype
  if isinstance(field_dtype , pd.core.dtypes.dtypes.DatetimeTZDtype):
    field_dtype = np.datetime64
    
  if not np.issubdtype(field_dtype , np.datetime64):
    df[fieldName] = field = pd.to_datetime(field , infer_datetime_format = True)
  targ_pre = re.sub('[Dd]ate$', '', fieldName)
  attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
          'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
  if time: attr = attr + ['Hour', 'Minute', 'Second']
  for n in attr: df[targ_pre + n] = getattr(field.dt, n.lower())
  df[targ_pre + 'Elapsed'] = field.astype(np.int64) // 10 ** 9
  if drop: df.drop(fieldName, axis=1, inplace=True)

In [0]:
add_datepart(weather , "Date" , drop = False)
add_datepart(googletrend , "Date" , drop = False)
add_datepart(train , "Date" , drop = False)
add_datepart(test ,"Date" , drop = False )

In [0]:
trend_de = googletrend[googletrend.file == 'Rossmann_DE']

In [21]:
#outer join followed by null-check
#we do outta join to because data might have unfilled values
store = join_df(store , store_states , "Store")
len(store[store.State.isnull()])

0

In [22]:
joined = join_df(train , store , "Store")
joined_test = join_df(test ,  store , "Store")
len(joined[joined.StoreType.isnull()]) , len(joined_test[joined_test.StoreType.isnull()]) 

(0, 0)

In [23]:
joined = join_df(joined, googletrend, ["State","Year", "Week"])
joined_test = join_df(joined_test, googletrend, ["State","Year", "Week"])
len(joined[joined.trend.isnull()]),len(joined_test[joined_test.trend.isnull()])

(995890, 41088)

In [0]:
joined = joined.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
joined_test = joined_test.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
len(joined[joined.trend_DE.isnull()]),len(joined_test[joined_test.trend_DE.isnull()])

In [0]:
joined = join_df(joined, weather, ["State","Date"])
joined_test = join_df(joined_test, weather, ["State","Date"])
len(joined[joined.Mean_TemperatureC.isnull()]),len(joined_test[joined_test.Mean_TemperatureC.isnull()])